# Parallel Processing

### Mutable vs Immutable Variables

In [4]:
def add1(x):
    x = x + 1
    
def addelement(y):
    y.append("NEW")

# External integers are immutable from within a function
x = 5
add1(x)
print(x)

# External lists and dictionaries are mutable from within a function
y = ["OLD","OLD"]
addelement(y)
print(y)

5
['OLD', 'OLD', 'NEW']


* Parallel processing works best with mutable variables since multiple processes need modify access to a variable at the same time

### Multithreading

In [7]:
import threading

class Counter():
    def __init__(self):
        self.count = 0
    def increment(self):
        self.count += 1
    def get_count(self):
        return self.count

def count_up_100000(counter):
    for i in range(100000):
        counter.increment()

counter = Counter()

# Initializing a thread object with a process: a function and its arguments
count_thread = threading.Thread(target=count_up_100000, args=[counter])

# Starting a thread's process
count_thread.start()

# Instructions to join the thread back to the main process
count_thread.join()

# Main thread continues only after all started threads have completed their processes and rejoined the main thread

after_join = counter.get_count()
print(after_join)

100000


#### Threading is Non-Deterministic
* Before thread is completed and `thread.join()` rejoins it back to the main thread, process completion is non-deterministic in nature

In [16]:
def conduct_trial():
    counter = Counter()
    count_thread = threading.Thread(target=count_up_100000, args=[counter])
    count_thread.start()
    return counter.get_count()

    # Not called, does not wait for thread process to complete
    # count_thread.join()
    
trial1 = conduct_trial()
trial2 = conduct_trial()
trial3 = conduct_trial()
print(trial1,trial2,trial3)

50939 27005 30535


#### Locking
* A lock is a way to conditionally block the execution of some threads.
* A lock is either __available__ or __aquired__
* Threads are only allowed to proceed if there are __available__ locks
* If all locks have been __aquired__ by other threads, a thread will be blocked until a lock is __available__
* A lock is like a constrained resource shared between threads

In [29]:
import threading

def count_up_100000(counter, lock):
    for i in range(10000):
        
        # Lock is only ever released to become available in counter increments of 10
        lock.acquire()
        for i in range(10):
            counter.increment()
        lock.release()

def conduct_trial():
    counter = Counter()
    
    # Initialize a Lock
    lock = threading.Lock()
    
    count_thread = threading.Thread(target=count_up_100000, args=[counter, lock])
    count_thread.start()
    
    # Only can run if a lock is available, grabs the counter at wherever count_thread is at
    lock.acquire()
    intermediate_value = counter.get_count()
    lock.release()
    
    # Allows count_thread to continue until is it completed
    count_thread.join()
    return intermediate_value

trial1 = conduct_trial()
print(trial1)
trial2 = conduct_trial()
print(trial2)
trial3 = conduct_trial()
print(trial3)

23250
48390
18460


### Two Threads Simultaneously

* Non Deterministic since counter.increment() is non-atomic, which means Thread 1 and Thread 2 could both be modifying and overwriting the values of the counter at the same time.

In [34]:
import threading

def count_up_100000(counter):
    for i in range(100000):
        counter.increment()

def conduct_trial():
    counter = Counter()
    count_thread1 = threading.Thread(target=count_up_100000, args=[counter])
    count_thread2 = threading.Thread(target=count_up_100000, args=[counter])

    count_thread1.start()
    count_thread2.start()

    # Join the threads here
    count_thread2.join()
    count_thread1.join()


    final_count = counter.get_count()
    return final_count

trial1 = conduct_trial()
print(trial1)
trial2 = conduct_trial()
print(trial2)
trial3 = conduct_trial()
print(trial3)

186668
200000
200000


* Atomic locking will prevent non-deterministic overwriting since only one thread can call counter.increment() at a time

In [ ]:
import threading

class Counter():
    def __init__(self):
        self.count = 0
    def increment(self):
        old_count = self.count
        self.count = old_count + 1
    def get_count(self):
        return self.count

def count_up_100000(counter, lock):
    for i in range(100000):
        lock.acquire()
        counter.increment()
        lock.release()
        
def conduct_trial():
    counter = Counter()
    
    lock = threading.Lock()
    
    count_thread1 = threading.Thread(target=count_up_100000, args=[counter,lock])
    count_thread2 = threading.Thread(target=count_up_100000, args=[counter,lock])

    count_thread1.start()
    count_thread2.start()

    count_thread1.join()
    count_thread2.join()

    final_count = counter.get_count()
    return final_count

trial1 = conduct_trial()
print(trial1)
trial2 = conduct_trial()
print(trial2)
trial3 = conduct_trial()
print(trial3)